In [26]:
import pandas as pd

In [27]:
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,15:30,Bundesliga,Matchweek 1,Sat,Home,W,3.0,2.0,RB Leipzig,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,18:30,Bundesliga,Matchweek 2,Sat,Away,W,3.0,0.0,M'Gladbach,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,15:30,Bundesliga,Matchweek 3,Sat,Home,W,5.0,1.0,Darmstadt 98,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,20:30,Bundesliga,Matchweek 4,Fri,Away,D,2.0,2.0,Bayern Munich,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,15:30,Bundesliga,Matchweek 5,Sun,Home,W,4.0,1.0,Heidenheim,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen


# Data preprocessing

## Get to know data

In [28]:
# drop unnecessary columns
matches_dropped = matches.drop(columns=['time', 'day', 'attendance', 'captain', 'formation', 'referee',
                                        'match report', 'notes'])
matches_dropped.head()

,date,comp,round,venue,result,gf,ga,opponent,xg,xga,...,tklw,blocks,int,clr,touches,att_y,succ,succ%,season,team
1,2023-08-19,Bundesliga,Matchweek 1,Home,W,3.0,2.0,RB Leipzig,1.4,1.6,...,11.0,14.0,8.0,15.0,574.0,22.0,11.0,50.0,2023-2024,Bayer Leverkusen
2,2023-08-26,Bundesliga,Matchweek 2,Away,W,3.0,0.0,M'Gladbach,2.7,0.9,...,11.0,9.0,9.0,11.0,854.0,16.0,7.0,43.8,2023-2024,Bayer Leverkusen
3,2023-09-02,Bundesliga,Matchweek 3,Home,W,5.0,1.0,Darmstadt 98,2.8,0.4,...,4.0,10.0,4.0,3.0,913.0,26.0,15.0,57.7,2023-2024,Bayer Leverkusen
4,2023-09-15,Bundesliga,Matchweek 4,Away,D,2.0,2.0,Bayern Munich,2.1,2.1,...,12.0,9.0,21.0,19.0,744.0,33.0,16.0,48.5,2023-2024,Bayer Leverkusen
6,2023-09-24,Bundesliga,Matchweek 5,Home,W,4.0,1.0,Heidenheim,3.4,0.5,...,10.0,11.0,4.0,12.0,964.0,23.0,14.0,60.9,2023-2024,Bayer Leverkusen


In [29]:
# shape
matches_dropped.shape

(2826, 39)

In [30]:
# take a look at first info 
matches_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2826 entries, 1 to 35
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      2826 non-null   object 
 1   comp      2826 non-null   object 
 2   round     2826 non-null   object 
 3   venue     2826 non-null   object 
 4   result    2826 non-null   object 
 5   gf        2826 non-null   float64
 6   ga        2826 non-null   float64
 7   opponent  2826 non-null   object 
 8   xg        2824 non-null   float64
 9   xga       2824 non-null   float64
 10  poss      2826 non-null   float64
 11  sh        2826 non-null   float64
 12  sot       2826 non-null   float64
 13  sot%      2825 non-null   float64
 14  saves     2826 non-null   float64
 15  save%     2757 non-null   float64
 16  stp       2824 non-null   float64
 17  stp%      2824 non-null   float64
 18  #opa      2824 non-null   float64
 19  cmp       2824 non-null   float64
 20  att_x     2824 non-null   float64
 21

In [31]:
# take a look at the columns
matches_dropped.columns

Index(['date', 'comp', 'round', 'venue', 'result', 'gf', 'ga', 'opponent',
       'xg', 'xga', 'poss', 'sh', 'sot', 'sot%', 'saves', 'save%', 'stp',
       'stp%', '#opa', 'cmp', 'att_x', 'cmp%', 'totdist', 'prgdist', 'ast',
       'kp', 'ppa', 'crspa', 'tkl', 'tklw', 'blocks', 'int', 'clr', 'touches',
       'att_y', 'succ', 'succ%', 'season', 'team'],
      dtype='object')

In [32]:
# dtypes
matches_dropped.dtypes

date         object
comp         object
round        object
venue        object
result       object
gf          float64
ga          float64
opponent     object
xg          float64
xga         float64
poss        float64
sh          float64
sot         float64
sot%        float64
saves       float64
save%       float64
stp         float64
stp%        float64
#opa        float64
cmp         float64
att_x       float64
cmp%        float64
totdist     float64
prgdist     float64
ast           int64
kp          float64
ppa         float64
crspa       float64
tkl         float64
tklw        float64
blocks      float64
int         float64
clr         float64
touches     float64
att_y       float64
succ        float64
succ%       float64
season       object
team         object
dtype: object

In [33]:
# some statistics summary
matches_dropped.describe()

,gf,ga,xg,xga,poss,sh,sot,sot%,saves,save%,...,crspa,tkl,tklw,blocks,int,clr,touches,att_y,succ,succ%
count,2826.000000,2826.000000,2824.000000,2824.000000,2826.000000,2826.000000,2826.000000,2825.000000,2826.000000,2757.000000,...,2824.000000,2824.000000,2826.000000,2824.000000,2826.000000,2824.000000,2824.000000,2824.000000,2824.000000,2824.000000
mean,1.567587,1.567587,1.478470,1.478470,50.000354,12.788747,4.426752,34.747221,3.029016,67.878527,...,2.113314,16.054887,9.554494,11.680595,10.667728,19.361544,615.812323,16.781161,8.875354,52.579533
std,1.350921,1.350921,0.855113,0.855113,11.126386,5.169717,2.546234,15.584948,2.038533,26.870918,...,1.763540,5.135310,3.594889,3.996274,4.343849,8.715438,122.447037,6.510156,4.391815,15.715098
min,0.000000,0.000000,0.000000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,...,0.000000,3.000000,0.000000,1.000000,0.000000,0.000000,321.000000,3.000000,0.000000,0.000000
25%,1.000000,1.000000,0.900000,0.900000,42.000000,9.000000,3.000000,25.000000,1.000000,50.000000,...,1.000000,12.000000,7.000000,9.000000,8.000000,13.000000,526.000000,12.000000,6.000000,42.100000
50%,1.000000,1.000000,1.300000,1.300000,50.000000,12.000000,4.000000,33.300000,3.000000,70.000000,...,2.000000,16.000000,9.000000,11.000000,10.000000,18.000000,603.000000,16.000000,8.000000,53.300000
75%,2.000000,2.000000,1.900000,1.900000,58.000000,16.000000,6.000000,44.400000,4.000000,88.900000,...,3.000000,19.000000,12.000000,14.000000,13.000000,24.000000,692.000000,21.000000,11.000000,63.200000
max,8.000000,8.000000,5.800000,5.800000,81.000000,34.000000,20.000000,100.000000,19.000000,100.000000,...,13.000000,34.000000,24.000000,28.000000,29.000000,62.000000,1101.000000,46.000000,28.000000,100.000000


## Data cleaning

### Duplicates

# EDA

# Feature engineering

In [25]:
# turn date column to datetime
matches['date'] = pd.to_datetime(matches['date'])